In [1]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Data preprocessing 

# Training Image preprocessing

In [ ]:
train_set = tf.keras.utils.image_dataset_from_directory(
    directory='train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)


# Validating Image Processing


In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    directory='valid',
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
   )

In [ ]:
train_set

In [ ]:
#To avoid Overshooting of Loss Function
#Choose small learning rate default 0.001 here we have taken 0.0001
#There may be chance of underfitting so increase number of neuron
#Add more Convolutional Layer to extract more feature from images there may be possibilty that model unable to capture relevant feature or model is confusing due to lack of feature so feed with more feature


# Building Model

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential 

In [ ]:
model=tf.keras.models.Sequential()

In [ ]:
##Building  Convolution layer

In [ ]:
# Add the first convolutional layer
model.add(Conv2D(filters=32, kernel_size=3,padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(filters=32, kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

In [ ]:
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3,  activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

In [ ]:
model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

In [ ]:
model.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=2, strides=2))

In [ ]:
model.add(Dropout(0.25))#To avoid Overfitting

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(units=1500,activation='relu'))

In [ ]:
model.add(Dropout(0.4))

In [ ]:
#Output Layer
model.add(Dense(units=38,activation='softmax'))

# Compiling Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# model.summary()

# Model training

In [ ]:
training_history=model.fit(x=train_set,validation_data=validation_set,epochs=10)


In [ ]:
#Model Evaluation on Training set
training_loss,train_acc=model.evaluate(train_set)

In [ ]:
print(training_loss,train_acc)

In [ ]:
#Model Evalutaion on Validation set
val_loss,val_acc=model.evaluate(validation_set)

In [ ]:
print(val_loss,val_acc)

# Model Saving

In [ ]:
model.save("trained_model.keras")

In [ ]:
training_history.history

In [ ]:
#Record History in json for future use
import json
with open("training_hist.json","w") as f:
    json.dump(training_history.history,f)

# Accuracy Visualization

In [ ]:
print(training_history.history['accuracy'])
epochs = range(1, 11)
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
epochs = range(1, 11)
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel("No of Epochs")
plt.ylabel("Acuuracy Result")
plt.title("Visualization of Accuracy Result")
plt.legend()
plt.show()

# Some other metrics for model evaluation

In [ ]:
class_name=validation_set.class_names
class_name

In [ ]:
test_set=tf.keras.utils.image_dataset_from_directory(
    directory='valid',
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
   )

In [ ]:
y_pred=model.predict(test_set)
y_pred

In [ ]:
predicted_categories=tf.argmax(y_pred,axis=1)

In [ ]:
predicted_categories

In [ ]:
true_categories=tf.concat([y for x,y in test_set],axis=0)
true_categories

In [ ]:
Y_true=tf.argmax(true_categories,axis=1)
Y_true

In [ ]:
# Precision measures the percentage of predictions made by the model that are correct. Recall measures the percentage of relevant data points that were correctly identified by the model.

# Precision Recall

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(Y_true,predicted_categories,target_names=class_name))

In [ ]:
cn=confusion_matrix(Y_true,predicted_categories)
cn.shape

# Confusion Matrix Visulaization

In [ ]:
plt.figure(figsize=(40,40))
sns.heatmap(cn,annot=True,annot_kws={'size':10})
plt.xlabel("Predicted Class",fontsize=25)
plt.ylabel("Actual Class",fontsize=25)
plt.title("Plant Disease Prediction Confusion Matrix",fontsize=25)
plt.show()